In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import rcParams
import lmfit
import matplotlib.image as mpimg
from PIL import Image
from pathlib import Path
from collections import OrderedDict
from lmfit.models import LorentzianModel
from scipy.optimize import curve_fit, OptimizeWarning

In [ ]:
# bokeh packages
from bokeh.io import output_file,show,output_notebook,push_notebook, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource,HoverTool,CategoricalColorMapper, Select, Slider
from bokeh.palettes import Category10
from bokeh.layouts import row,column,gridplot,widgetbox
from bokeh.models.widgets import Tabs,Panel
output_notebook()

In [ ]:
plt.style.use('classic')
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
rcParams.update({'font.size': 22})
rcParams['xtick.direction'] = 'in'
rcParams['ytick.direction'] = 'in'
rcParams['xtick.top'] = True
rcParams['ytick.right'] = True
rcParams['xtick.major.size'] = 10
rcParams['ytick.major.size'] = 10
rcParams['xtick.minor.size'] = 5
rcParams['ytick.minor.size'] = 5
rcParams['xtick.minor.visible'] = True
rcParams['ytick.minor.visible'] = True

In [ ]:
#genfromtxt(fname, dtype, comments, delimiter, skip_header, skip_footer, 
    #          converters, missing_values, filling_values, usecols, names, excludelist, deletechars, 
    #          replace_space, autostrip, case_sensitive, defaultfmt, unpack, usemask, loose, 
    #          invalid_raise, max_rows, encoding)  
    
#loading data using numpy
def load_data(filename):
    return np.genfromtxt(filename, delimiter='', skip_header=5)

#selecting the range 
def extract_peak(data,xrange):
    idx = np.logical_and(xrange[0] < data[:,0],data[:,0] < xrange[1])
    return data[idx,:]

In [ ]:
#making dictionary for selected folder
def load_folder(folder,name):
    temp = {}
    for file in folder.glob(name):
        d = load_data(file)
        k = file.name[1:12]
        temp[k] = d
    return OrderedDict(sorted(temp.items()))

#for sector cut files which has header with 14 rows
def load_folder1(folder,name):
    temp = {}
    for file in folder.glob(name):
        d = np.genfromtxt(file, delimiter='', skip_header=14)
        k = file.name[0:12]
        temp[k] = d
    return OrderedDict(sorted(temp.items()))

In [ ]:
#plotting using Bokeh
def bokeh_plot(data,x_range=[0.001, 0.07], y_range=[0.0001, 3000] ):
    f = figure(x_range=x_range, y_range=y_range,x_axis_type="log",y_axis_type="log",
               x_axis_label='Q', y_axis_label="Intensity") 
    for k,color in zip(data,Category10[10]):
        d = extract_peak(data[k],x_range)
        f.circle(d[:,0],d[:,1],color=color, legend_label=k,size=6,alpha=1,
                     hover_color='purple', hover_alpha=0.6)
        hover = HoverTool(tooltips=[ ('Q', '@x'), ('I', '@y')], mode='mouse')
        f.add_tools(hover)
    show(f)

In [ ]:
def tabs_plot(data, x_range=[0.001, 0.07], y_range=[0.001, 3000]):
    tabs=[]
    for k in data:
        d = data[k]
        f=figure(x_range=x_range, y_range=y_range,x_axis_type="log", y_axis_type="log",
                 x_axis_label='Q', y_axis_label="Intensity")
        x=d[:,0]
        y=d[:,1]
        f.circle(x, y, size=10,fill_color='red',alpha=1,
                     hover_color='purple', hover_alpha=0.6)
        hover = HoverTool(tooltips=[ ('Q', '@x'), ('I', '@y')], mode='mouse')
        f.add_tools(hover)
        tab = Panel(child=f, title=k[:3])
        tabs.append(tab)
    layout = Tabs(tabs=tabs)
    show(layout)

In [ ]:
#plottting all fields of same temp in one tab
def all_tab_plot(data, folder, x_range=[0.001, 0.07], y_range=[0.001, 3000]): 
    data1=[load_folder(folder, d) for d in data]
    temper=['18K','20K','22K','24K','25K','26K','30K','75K','200K']
    tabs=[]
    for temp, i in zip(data1, range(9)):
        f=figure(x_range=x_range, y_range=y_range, x_axis_type="log", y_axis_type="log", 
                 y_axis_label=' Intensity', x_axis_label='Q')
        for k, color in zip(temp, Category10[10]):
            d = temp[k]
            x=d[:,0]
            y=d[:,1]
            f.circle(x,y, color=color,legend_label=k, alpha=1,size=10,
                     hover_color='purple', hover_alpha=0.6)
        hover = HoverTool(tooltips=[ ('Q', '@x'), ('I', '@y')], mode='mouse')
        f.add_tools(hover)
        tab = Panel(child=f, title=temper[i])
        tabs.append(tab)
    layout = Tabs(tabs=tabs)
    show(layout)

In [ ]:
all_tab_plot(vert_comb_low, vfolder)

In [ ]:
#plottting all sector cut of same temp in one tab
def all_tab_secplot(data, folder, x_range=[0.001, 0.01], y_range=[0.1, 3000]): 
    data1=[load_folder1(folder, d) for d in data]
    temper=['18K','20K','24K','26K','30K']
    tabs=[]
    for temp, i in zip(data1, range(5)):
        f=figure(x_range=x_range, y_range=y_range,x_axis_type="log", y_axis_type="log", 
                 y_axis_label=' Intensity', x_axis_label='Q')
        for k, color in zip(temp, Category10[10]):
            d = temp[k]
            x=d[:,0]
            y=d[:,1]
            f.circle(x,y, color=color,legend_label=k, alpha=1,size=10,
                     hover_color='purple', hover_alpha=0.6)
        hover = HoverTool(tooltips=[ ('Q', '@x'), ('I', '@y')], mode='mouse')
        f.add_tools(hover)
        tab = Panel(child=f, title=temper[i])
        tabs.append(tab)
    layout = Tabs(tabs=tabs)
    show(layout)

In [ ]:
all

In [ ]:
vfolder=Path.cwd()/'vertical'   #folder for vertical cut
hfolder=Path.cwd()/'horizantal'  #folder for horizontal cut
sec_folder=Path.cwd()/'Sunil_Sinha_Nov2019' # folder for sector cut(0,30,60,90,120,150 degree)

In [ ]:
vert_comb_low=load_folder(vfolder,'')  #vertical cut for all temp at 700 Oe
vert_comb_1=load_folder(vfolder,'*1*.ABS')   #vertical cut for all temp at 1 kOe
vert_comb_2=load_folder(vfolder,'*.ABS')  #vertical cut for all temp at 2 kOe
vert_comb=[vert_comb_low, vert_comb_1,vert_comb_2]


In [ ]:
horz_comb_low=load_folder(hfolder,'H*comb*low*.ABS') #horizontal cut for all temp at 700 Oe
horz_comb_1=load_folder(hfolder,'')   #horizontal cut for all temp at 1 kOe
horz_comb_2=load_folder(hfolder,'') #horizontal cut for all temp at 2 kOe


# Fitting using lm

In [ ]:
#defining fit function using lmfit
def fit_qi_lm(data, model='Lort_Lort2', **kwargs):
    A    = kwargs.pop('A',  1.0)
    A2   = kwargs.pop('A2',  1.0)
    L1   = kwargs.pop('L1',  1.0)
    L2   = kwargs.pop('L2',  1.0)
    B    = kwargs.pop('B',  1.0)
    t0   = kwargs.pop('t0', 1.0)
    i    =kwargs.pop('i', 1.0)
    m    =kwargs.pop('m', 1.0)
    init_par = kwargs.pop('init_par', None)
    
    function_library = {
        'Lort'      : lambda x, L1, A, B: A/(x**2+(1/L1**2))+B,
        'Lort2'     : lambda x, L1, A, B: (A/(x**2+(1/L1**2))**2)+B,
        'Lort_Lort2': lambda x, L1, L2, A, A2, B: (A/(x**2+(1/L1**2)))+(A2/(x**2+(1/L2**2))**2)+B,
        'Guiner'    : lambda x, i, m, A, B: (np.log(i) - m*x) +B,
        'Exp'       : lambda t, t0, A:      A*np.exp(-(t/t0)),         # simple exponential function
        'Power'     : lambda x, b,  A:      A*x**b,                        # power law
        'Linear'    : lambda x, x0, A:      A*(x-x0),                      # linear function
        'Constant'  : lambda x, A:          A*np.ones_like(x),             # constant
        
    }
    q  = data[:,0]
    I  = data[:,1]
    Ierr =data[:,2]
    
    func = function_library.get(model, None)
    if func is None: raise RuntimeError(f"model {model} is invalid")
    fit_model = lmfit.Model(func,independent_vars='x',nan_policy='omit')
  
    if model.startswith('Lort_Lort2'):
        fit_pars = fit_model.make_params(A=8e-6, A2=5e-9, L1=1e7, L2=2000,B=1000)
        fit_pars['B'].set(min=0)  #only positive values
        fit_pars['A'].set(vary=True,min=0)
        fit_pars['A2'].vary=False
        fit_pars['L1'].min=0
        fit_pars['L2'].set(min=0)
        
    elif model.startswith('Lort2'):
        fit_pars = fit_model.make_params(A=1e-8,L1=1000,B=1)
        fit_pars['B'].set(min=0)
        fit_pars['A'].set(vary=True,max=0.0000001)
        fit_pars['L1'].set(min=0)
        
    elif model.startswith('Lort'):
        fit_pars = fit_model.make_params(A=1e-7,L1=100,B=1)
        fit_pars['B'].set(min=0)
        fit_pars['A'].set(vary=True,max=0.0000001)
        fit_pars['L1'].set(min=0)
    
    elif model.startswith('Guiner'):
        fit_pars=fit_model.make_params(i=0.003,B=0)
        fit_pars['m'].value = 0.003
        fit_pars['i'].set(min=0.01)
        fit_pars['B'].set(min=0,max=100)
        
    else:
        fit_pars = fit_model.make_params(t0=1, beta=1, A=1, B=1)
        fit_pars['B'].set(min=0)  #only positive values
        fit_pars['t0'].set(vary=True,min=0)
        fit_pars['beta'].min=0
        fit_pars['A'].set(min=0)
        
    result = fit_model.fit(I,fit_pars,x=q,method='least_squares')
    return result
    

In [ ]:
#defining function for plotting and data
def lm_fit(data, **kwargs):
    position=kwargs.pop('position',[0.9,0.05])
    model = kwargs.pop('model', 'Lort_Lort2') # fit model (see fit_qi)
    name=kwargs.pop('name','name')
    mpars = kwargs.pop('model_pars', {} )   # model parameters
    plot_summary = kwargs.pop('plot_summary', True)
    plot    = kwargs.pop('plot', True)

    r = []; ks = []
    if plot:
        plt.figure(figsize=(15,15))

    samples=data
    for sample in samples:
        label  = sample[:4]  
        d = extract_peak(samples[sample],position)
        q=d[:,0]
        I=d[:,1]
        Ierr=d[:,2]  
                
        if plot:
            #p = plt.scatter(q, I, label=label,s=8)
            p = plt.errorbar(q, I, yerr=Ierr,fmt='s', label=label)
            acolor = p[0].get_color()   

        result=fit_qi_lm(d, model=model)
       
        fit_plot = result.best_fit

        if plot:
            plt.plot(q, fit_plot, '--',linewidth=3, color=acolor)
        ks.append(sample)
        r.append(result.best_values)

    if plot:
            plt.xscale('log')
            plt.yscale('log')
            #plt.yscale('log')
            plt.xlabel(r'q $\AA^{-1}$')
            plt.ylabel('Intensity')
            plt.grid(which='both')
            plt.legend(loc=0)
    plt.show()
    df=pd.DataFrame(r,index=ks)
    
    return df

    
                  

In [ ]:
lm_fit(vert_comb_low, model='Lort_Lort2', position=[0.001,0.07])

# Fitting with normal method

In [ ]:
#defining fit function 
def fits_qi(data, model='Lort_Lort2', **kwargs):
    A    = kwargs.pop('A',  1.0)
    A2   = kwargs.pop('A2',  1.0)
    L1   = kwargs.pop('L1',  1.0)
    L2   = kwargs.pop('L2',  1.0)
    B    = kwargs.pop('B',  1.0)
    t0   = kwargs.pop('t0', 1.0)
    i    =kwargs.pop('i', 1.0)
    m    =kwargs.pop('m', 1.0)
    init_par = kwargs.pop('init_par', None)
    
    function_library = {
        'Lort'      : lambda x, L1, A, B: A/(x**2+(1/L1**2))+B,
        'Lort2'     : lambda x, L1, A, B: (A/(x**2+(1/L1**2))**2)+B,
        'Lort_Lort2': lambda x, L1, L2, A, A2, B: (A/(x**2+(1/L1**2)))+(A2/(x**2+(1/L2**2))**2)+B,
        'Guiner'    : lambda x, I, m, A: (np.log(i) - m*x) +B,
        'Exp'       : lambda t, t0, A:      A*np.exp(-(t/t0)),         # simple exponential function
        'Power'     : lambda x, b,  A:      A*x**b,                        # power law
        'Linear'    : lambda x, x0, A:      A*(x-x0),                      # linear function
        'Constant'  : lambda x, A:          A*np.ones_like(x),             # constant
        
    }
    q  = data[:,0]
    I  = data[:,1]
    Ierr =data[:,2]

    func = function_library.get(model, None)
    if func is None: raise RuntimeError(f"model {model} is invalid")

    best_values, covar = curve_fit(func, q, I, 
                           sigma=Ierr, p0=init_par,
                             maxfev=10000, ftol=1e-5)
    #popt gives all the best value parameters in order of function
    perr = np.sqrt(np.diag(covar))
    fit_func = lambda x : func(x, *best_values)
    chisq = np.sum(((fit_func(q) - I)/Ierr)**2)
    chisq = chisq/(len(I) - len(best_values))
    return (fit_func, chisq, best_values,perr)

In [ ]:
#defining function for plotting and data
def normal_fit(data, **kwargs):
    position=kwargs.pop('position',[0.001,0.01])
    model = kwargs.pop('model', 'Lort_Lort2') # fit model (see fit_qi)
    mpars = kwargs.pop('model_pars', {} )   # model parameters
    plot_summary = kwargs.pop('plot_summary', True)
    plot    = kwargs.pop('plot', True)

    r = []; ks = []
    if plot:
        plt.figure(figsize=(10,10))
 
    samples=data
    for sample in samples:
        label  = sample[:4]  
        d = extract_peak(samples[sample],position)
        q=d[:,0]
        I=d[:,1]
        Ierr=d[:,2]  
                
        if plot:
            p = plt.scatter(q, I, label=label, s=2)
            
            
        fit_func, chiq, best_values,perr = fits_qi(d, model=model)
        if plot:
            ftau = np.logspace(np.log10(min(q)), np.log10(max(q)))
            fsqt = fit_func(ftau)
            plt.plot(ftau, fsqt, '--',linewidth=3) 
        ks.append(sample)
        r.append(best_values)


    if plot:
            #plt.xscale('log')
            #plt.yscale('log')
            plt.xlabel(r'q $\AA^{-1}$')
            plt.ylabel('Intensity')
            plt.title(f"{label[:2]}, {model}")
            plt.grid(which='both')
            plt.legend(loc=0)
    plt.show()
    df=pd.DataFrame(r,index=ks)
    return df

                  

In [ ]:
normal_fit(vert_comb_1)

In [ ]:
#plotting high temp, subs with high temp and without subsatation 
v30=load_data(Path.cwd()/'vertical/V30K_comb_lowfield.ABS')
v30_ws=load_data(Path.cwd()/'vertical/V30_COMBO_70Oe(without Ba.ABS')
v75=load_data(Path.cwd()/'vertical/V75K_comb_lowfield.ABS')
fig, ax= plt.subplots(figsize=(15,10))
ax.plot(v30[:,0], v30[:,1],'or',label='30K-75K')
ax.plot(v30_ws[:,0], v30_ws[:,1],'ob', label='30K')
ax.plot(v75[:,0], v75[:,1],'og', label='75K')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlim(0.001, 0.1)
ax.legend()

In [ ]:
v_26k=load_folder(vfolder,'V26K_15m*.ABS')
bokeh_plot(v_26k)

In [ ]:
vert_comb=['V18K*comb*.ABS','V20K*comb*.ABS','V22K*comb*.ABS','V24K*comb*.ABS','V25K*comb*.ABS',
           'V26K*comb*.ABS','V30K*comb*.ABS','V75K*comb*.ABS', 'V200K*comb*.ABS']
horz_comb=['H18K*comb*.ABS','H20K*comb*.ABS','H22K*comb*.ABS','H24K*comb*.ABS','H25K*comb*.ABS',
           'H26K*comb*.ABS','H30K*comb*.ABS','H75K*comb*.ABS','H200K*comb*.ABS']

all_tab_plot(data=vert_comb, folder=vfolder)
#all_tab_plot(horz_comb,folder=hfolder)

In [ ]:
all_tab_plot(data=horz_comb, folder=hfolder)

In [ ]:
#for sector plot
sec_15m=['15m_18K*deg*.ASC','15m_20K*deg*.ASC','15m_24K*deg*.ASC','15m_26K*deg*.ASC','15m_30K*deg*.ASC']

all_tab_secplot(data=sec_15m, folder=sec_folder)

In [ ]:
#to import multiple figures
# single figures without averaging
igor_2dfig=Path.cwd()/'2d_figures'

        
def twod_figure(filename: str=None) -> None:
    """
    View multiple images stored in files, stacking vertically

    Arguments:
        filename: str - path to filename containing image
    """
    image = Image.open(filename)    #from PIL import Image
    #image = mpimg.imread(filename)
    #plt.figure()
    #plt.imshow(image1)
    display(image)

In [ ]:
for file in igor_2dfig.glob('*.png'):
    twod_figure(file)

In [ ]:
for file in igor_2dfig.glob('s*.png'):
    twod_figure(file)

In [ ]:
#Lorentzian square
def lort(x,amp,L1,background=0):
    lo=amp/(x**2+(1/L1**2))
    return lo + background 

def lort2(x,amp,L1,background=0):
    lo2=amp/(x**2+(1/L1**2))**2
    return lo2 + background

#Lorentzian and lorentzian square
def lort_lort2(x,amp1,amp2,L1,L2,background=0):
    lo=amp1/(x**2+(1/L1**2))
    lo2=amp2/(x**2+(1/L2**2))**2
    return lo+ lo2 + background

#Guinier Approximation
#ln(q)=lnI0-Rg**2/3*q**2
def guin(x, slope, intensity, background=0):   
    return (np.log(intensity) - slope*x) +background

In [ ]:
#using lmfit- assigning model for lorentzian and lorentziansquare  
lomodel = lmfit.Model(lort,independent_vars='x',nan_policy='omit')
lo2model = lmfit.Model(lort2,independent_vars='x',nan_policy='omit')
lolo2model = lmfit.Model(lort_lort2,independent_vars='x',nan_policy='omit')
guinmodel = lmfit.Model(guin,independent_vars='x',nan_policy='omit')

In [ ]:
#For lorentzian  changing parameters like initial, max, and min value
lopars = lomodel.make_params(amp=1e-7,L1=100,background=1)
lopars['background'].set(min=0)
lopars['amp'].set(vary=True,max=0.0000001)
lopars['L1'].set(min=0)

#For lorentzian square- changing parameters like initial, max, and min value
lo2pars = lo2model.make_params(amp=5e-8,L1=1000,background=1)
lo2pars['background'].set(min=0)
lo2pars['amp'].set(vary=True)
lo2pars['L1'].set(min=0)

#for lorentzian_lorentzian2-changing parameters initial, max, and min value
lolo2pars = lolo2model.make_params(amp1=8e-6, amp2=5e-9, L1=1e7, L2=2000,background=1000)
lolo2pars['background'].set(min=0)  #only positive values
lolo2pars['amp1'].set(vary=True,min=0)
lolo2pars['amp2'].vary=False
lolo2pars['L1'].min=0
lolo2pars['L2'].set(min=0)

#changing parameters initial, max, and min value
guinpars=guinmodel.make_params(intensity=0.003,background=0)
guinpars['slope'].value = 0.003
guinpars['intensity'].set(min=0.01)
guinpars['background'].set(min=0,max=100)


In [ ]:
lmodel=[guinmodel,lomodel, lo2model,lolo2model]
lpars=[guinpars, lopars, lo2pars, lolo2pars]
#function for fitting
def all_fit(data,position=[0.0001,0.07],lort=False, lort2=False, lort_lort2=False):
    r = []; ks = []
    for k in data:
        sample = data[k]
        d = extract_peak(sample,position)
        #height = d[:,1].max()/100
        if lort:
            fit = lmodel[1].fit(d[:,1],lpars[1],x=d[:,0],method='least_squares')
            fit.plot()
            plt.title(f'{k}')
            plt.xlabel('q')
            plt.ylabel('I')
            plt.xscale("log")
            plt.yscale("log")  
        elif lort2:
            fit = lmodel[2].fit(d[:,1],lpars[2],x=d[:,0],method='least_squares')
            fit.plot()
            plt.title(f'{k}')
            plt.xlabel('q')
            plt.ylabel('I')
            plt.xscale("log")
            plt.yscale("log")
        elif lort_lort2 :
            fit = lmodel[3].fit(d[:,1],lpars[3],x=d[:,0],method='least_squares')
            fit.plot()
            plt.title(f'{k}')
            plt.xlabel('q')
            plt.ylabel('I')
            plt.xscale("log")
            plt.yscale("log")
        else:
            fit =lmodel[0].fit((np.log(d[:,1])),lpars[0],x=(d[:,0]*d[:,0]),method='powell')
            fit.plot()
            plt.title(f'{k}')
            plt.xlabel('q2')
            plt.ylabel('ln(I)')
        ks.append(k)
        r.append(fit.best_values)
        df=pd.DataFrame(r,index=ks)
        
        if not (lort or lort2 or lort_lort2): #for guiner only
            df['size_nm']=(np.sqrt(3*df['slope']))/10
    return df